# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [ ]:
!cat publications.tsv

: 

## Import pandas

We are using the very handy pandas library for dataframes.

In [130]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [144]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,coauthors,journal,venue,excerpt,url_slug,paper_url,slides_url,category
0,2023-08-01,Individuals and Organizations as Sources of St...,Jonas Hjort and David Szakonyi,American Economic Review,"113(8), 2121-2167",Bureaucrats implement policy. How important ar...,bhs_state_effectiveness,http://michaelcbest.github.io/files/BestHjortS...,NaN,published
1,2021-08-19,The Allocation of Authority in Organizations: ...,"Oriana Bandiera, Adnan Khan, and Andrea Prat",Quarterly Journal of Economics,"136(4), 2195--2242",We design a field experiment to study how the ...,bbkp_procurement,http://michaelcbest.github.io/files/BBKP_Autho...,NaN,published
2,2017-06-24,Housing Market Responses to Transaction Taxes:...,Henrik Kleven,Review of Economic Studies,"85(1), 157-193",We investigate housing market responses to tra...,bk_stampduty,http://michaelcbest.github.io/files/best-kleve...,NaN,published
3,2020-03-01,Estimating the Elasticity of Intertemporal Sub...,"James Cloyne, Ethan Elzetzki, and Henrik Kleven",Review of Economic Studies,"87(2), 656-690",Using a novel source of quasi-experimental var...,bcik_mortgagenotches,http://michaelcbest.github.io/files/BCIK_Mortg...,NaN,published
4,2015-12-01,Production versus Revenue Efficiency with Limi...,"Anne Brockmeyer, Henrik Kleven, Johannes Spinn...",Journal of Political Economy,"123(6), 1311-1355","To fight evasion, many developing countries us...",bbksw_prodrevefficiency,http://michaelcbest.github.io/files/BestEtAl_P...,NaN,published
5,2014-09-30,Salary Misreporting and the Role of Firms in W...,NaN,NaN,NaN,This paper exploits employee-employer matched ...,salarymisreporting,http://michaelcbest.github.io/files/Best_Pakis...,NaN,working
6,2013-08-31,Optimal Income Taxation with Career Effects of...,Henrik Kleven,NaN,NaN,The literature on optimal income taxation assu...,taxcareerconcerns,http://michaelcbest.github.io/files/best-kleve...,NaN,working
7,2023-01-01,Sanitation and property tax compliance: Analyz...,"Evan Plous Kresch, Mark Walker, Fran&ccedil;oi...",Joural of Development Economics,"160, 102954",This paper investigates the role that sanitati...,kwbgn_sanitation,http://michaelcbest.github.io/files/KreschWalk...,NaN,published
8,2023-07-31,Government Analytics Using Administrative Case...,"Alessandra Fenizia, and Adnan Khan",Chapter 15 in The Government Analytics Handboo...,edited by Daniel Rogger and Christian Schuster...,Measuring the performance of government agenci...,bfk_analytics,http://michaelcbest.github.io/files/GAH-CHAPTE...,NaN,bkchap
9,2024-10-01,Greener on the Other Side,person1,NaN,NaN,NaN,bcgknz_inequity,http://michaelcbest.github.io/files/GAH-CHAPTE...,NaN,wip


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [142]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [145]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""

    if len(str(item.coauthors)) > 3:
        md += """\ncoauthors: """ + item.coauthors

    md += """\ncategory: """ + item.category
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '<details markdown=block>" + "\n<summary markdown=span> *Abstract*</summary> \n" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    if len(str(item.journal)) > 3:
        md += "\njournal: '" + str(item.journal) + "'"

    if len(str(item.venue)) > 3:
        md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    #md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page

    if len(str(item.coauthors)) > 3:
        md += "\n(with " + item.coauthors + ")\n"

    if len(str(item.excerpt)) > 5:
        md += "\n \n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [146]:
!ls ../_publications/

2013-08-31-taxcareerconcerns.md       2021-08-19-bbkp_procurement.md
2014-09-30-salarymisreporting.md      2023-01-01-kwbgn_sanitation.md
2015-12-01-bbksw_prodrevefficiency.md 2023-07-31-bfk_analytics.md
2017-06-24-bk_stampduty.md            2023-08-01-bhs_state_effectiveness.md
2020-03-01-bcik_mortgagenotches.md    2024-10-01-bcgknz_inequity.md


In [147]:
!cat ../_publications/2023-08-01-paper-title-number-1.md

cat: ../_publications/2023-08-01-paper-title-number-1.md: No such file or directory
